# Compare 2 (Excel) Datasets & Show The Difference 🧐

## Load Dependencies

**Requirements:**<br>
`openpyxl==3.0.9`<br>
`pandas==1.3.5`<br>
`xlwings==0.25.3`<br>

<p style="background:black">
<code style="background:black;color:white">C:\Users\YOUR_USERNAME> pip install pandas openpyxl xlwings
</code>
</p>

In [1]:
from pathlib import Path  # Core Python Module

import pandas as pd  # pip install pandas openpyxl
import xlwings as xw  # pip install xlwings

## Data with the same shape 🏄‍♂️

### Define Filepath

In [2]:
initial_version = Path.cwd() / "Same_Shape" / "Arrival_Dates.xlsx"
updated_version = Path.cwd() / "Same_Shape" / "Arrival_Dates_Final.xlsx"

### Load the DataFrames

In [3]:
df_initial = pd.read_excel(initial_version)
df_initial.head(3)

,First Name,Last Name,Arrival Date
0,Carlos,Sullivan,2022-10-02
1,Jeffrey,Giles,2022-04-25
2,Mckenzie,Perkins,2022-04-13


In [4]:
df_update = pd.read_excel(updated_version)
df_update.head(3)

,First Name,Last Name,Arrival Date
0,Carlos,Sullivan,2022-10-02
1,Jeffrey,Giles,2022-04-25
2,Mckenzie,Perkins,2022-04-13


---

**Pandas `compare` method**<br>
Find the docs here: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.compare.html

<div class="alert alert-block alert-warning">
<b>Note:</b> the method can only compare identically-labeled DataFrame objects, this means DataFrames with identical row and column labels.
</div>

In [5]:
df_initial.shape

(100, 3)

In [6]:
df_update.shape

(100, 3)

In [7]:
df_initial.shape == df_update.shape

True

### Align the differences on columns

In [8]:
diff = df_update.compare(df_initial, align_axis=1)
diff
# self = updated_version
# other = initial_version

First Name       Arrival Date           
         self other         self      other
10       Troi  Troy          NaT        NaT
21        NaN   NaN   2022-12-10 2022-03-23
56        NaN   NaN   2022-08-18 2022-02-01
85        NaN   NaN   2022-07-26 2022-10-24
99        NaN   NaN   2022-12-03 2022-07-09

### Stack the differences on rows

In [9]:
diff = df_update.compare(df_initial, align_axis=0)
diff

First Name Arrival Date
10 self        Troi          NaT
   other       Troy          NaT
21 self         NaN   2022-12-10
   other        NaN   2022-03-23
56 self         NaN   2022-08-18
   other        NaN   2022-02-01
85 self         NaN   2022-07-26
   other        NaN   2022-10-24
99 self         NaN   2022-12-03
   other        NaN   2022-07-09

### Keep all original rows and columns

In [10]:
diff = df_update.compare(df_initial, keep_shape=True, keep_equal=False)
diff

First Name       Last Name       Arrival Date           
         self other      self other         self      other
0         NaN   NaN       NaN   NaN          NaT        NaT
1         NaN   NaN       NaN   NaN          NaT        NaT
2         NaN   NaN       NaN   NaN          NaT        NaT
3         NaN   NaN       NaN   NaN          NaT        NaT
4         NaN   NaN       NaN   NaN          NaT        NaT
..        ...   ...       ...   ...          ...        ...
95        NaN   NaN       NaN   NaN          NaT        NaT
96        NaN   NaN       NaN   NaN          NaT        NaT
97        NaN   NaN       NaN   NaN          NaT        NaT
98        NaN   NaN       NaN   NaN          NaT        NaT
99        NaN   NaN       NaN   NaN   2022-12-03 2022-07-09

[100 rows x 6 columns]

### Keep all original rows and columns and also all original values

In [11]:
diff = df_update.compare(df_initial, keep_shape=True, keep_equal=True)
diff

First Name             Last Name             Arrival Date           
         self     other        self       other         self      other
0      Carlos    Carlos    Sullivan    Sullivan   2022-10-02 2022-10-02
1     Jeffrey   Jeffrey       Giles       Giles   2022-04-25 2022-04-25
2    Mckenzie  Mckenzie     Perkins     Perkins   2022-04-13 2022-04-13
3        Mary      Mary    Marshall    Marshall   2022-09-02 2022-09-02
4     Kathryn   Kathryn       Burch       Burch   2022-02-11 2022-02-11
..        ...       ...         ...         ...          ...        ...
95     Daniel    Daniel        Wall        Wall   2022-12-03 2022-12-03
96     Pamela    Pamela  Mclaughlin  Mclaughlin   2022-04-14 2022-04-14
97     Carmen    Carmen    Williams    Williams   2022-12-15 2022-12-15
98      Laura     Laura      Rogers      Rogers   2022-09-06 2022-09-06
99      David     David       Stout       Stout   2022-12-03 2022-07-09

[100 rows x 6 columns]

### Export difference to Excel 📥

In [12]:
diff = df_update.compare(df_initial, align_axis=1)
diff.to_excel(Path.cwd() / "Same_Shape" /"Difference.xlsx")

### [BONUS] Highlight the difference 🔥

In [13]:
with xw.App(visible=False) as app:
    initial_wb = app.books.open(initial_version)
    initial_ws = initial_wb.sheets(1)

    updated_wb = app.books.open(updated_version)
    updated_ws = updated_wb.sheets(1)

    for cell in updated_ws.used_range:
        old_value = initial_ws.range((cell.row, cell.column)).value
        if cell.value != old_value:
            cell.api.AddComment(f"Value from {initial_wb.name}: {old_value}")  # WARNING: Platform specific (!)
            cell.color = (255, 71, 76)  # light red

    updated_wb.save(Path.cwd() / "Same_Shape" / "Difference_Highlighted.xlsx")

## Data with different shape 🚨

### Load the DataFrames

In [14]:
initial_version = Path.cwd() / "Different_Shape" / "Arrival_Dates.xlsx"
updated_version = Path.cwd() / "Different_Shape" / "Arrival_Dates_Final.xlsx"

### Check shape & data

In [15]:
df_initial = pd.read_excel(initial_version)
df_initial.shape

(100, 3)

In [16]:
df_update = pd.read_excel(updated_version)
df_update.shape

(102, 3)

In [17]:
df_initial.shape == df_update.shape

False

### Show the difference by merging both DataFrames

In [18]:
# We need the index information to highlight the rows in Excel
df_update = df_update.reset_index()
df_update.head(3)

,index,First Name,Last Name,Arrival Date
0,0,Carlos,Sullivan,2022-10-02
1,1,Jeffrey,Giles,2022-04-25
2,2,Mckenzie,Perkins,2022-04-13


![PandasMerge](Pandas_Merge.png)

In [19]:
# Merge dataframes and add inidactor column
df_diff = pd.merge(df_initial, df_update, how="outer", indicator="Exist")
df_diff

,First Name,Last Name,Arrival Date,index,Exist
0,Carlos,Sullivan,2022-10-02,0.0,both
1,Jeffrey,Giles,2022-04-25,1.0,both
2,Mckenzie,Perkins,2022-04-13,2.0,both
3,Mary,Marshall,2022-09-02,3.0,both
4,Kathryn,Burch,2022-02-11,4.0,both
...,...,...,...,...,...
102,NaN,NaN,NaT,18.0,right_only
103,Kevin,Elliott,2022-12-10,23.0,right_only
104,Kathy,Ortiz,2022-08-18,58.0,right_only
105,Michael,Davis,2022-07-26,87.0,right_only


In [20]:
# Show only the differnce
df_diff = df_diff.query("Exist != 'both'")
df_diff

,First Name,Last Name,Arrival Date,index,Exist
10,Troy,Clay,2022-05-12,NaN,left_only
21,Kevin,Elliott,2022-03-23,NaN,left_only
56,Kathy,Ortiz,2022-02-01,NaN,left_only
85,Michael,Davis,2022-10-24,NaN,left_only
99,David,Stout,2022-07-09,NaN,left_only
100,Peter,Parker,2022-02-11,8.0,right_only
101,Troi,Clay,2022-05-12,11.0,right_only
102,NaN,NaN,NaT,18.0,right_only
103,Kevin,Elliott,2022-12-10,23.0,right_only
104,Kathy,Ortiz,2022-08-18,58.0,right_only


### [BONUS] Highlight the difference 🔥

In [21]:
# Show only the data we want to highlight
df_highlight = df_diff.query("Exist == 'right_only'")
df_highlight

,First Name,Last Name,Arrival Date,index,Exist
100,Peter,Parker,2022-02-11,8.0,right_only
101,Troi,Clay,2022-05-12,11.0,right_only
102,NaN,NaN,NaT,18.0,right_only
103,Kevin,Elliott,2022-12-10,23.0,right_only
104,Kathy,Ortiz,2022-08-18,58.0,right_only
105,Michael,Davis,2022-07-26,87.0,right_only
106,David,Stout,2022-12-03,101.0,right_only


In [22]:
# Get the row numbers we want to highlight in Excel
highlight_rows = df_highlight['index'].tolist()
highlight_rows

[8.0, 11.0, 18.0, 23.0, 58.0, 87.0, 101.0]

In [23]:
# Convert floats to integers
highlight_rows = [int(row) for row in highlight_rows]
highlight_rows

[8, 11, 18, 23, 58, 87, 101]

In [24]:
# pandas index starts at 0
# Excel data (w/o header) starts from row 2
first_row_in_excel = 2

highlight_rows = [x + first_row_in_excel for x in highlight_rows]
highlight_rows

[10, 13, 20, 25, 60, 89, 103]

#### Highlight the rows in Excel

In [25]:
with xw.App(visible=False) as app:
    updated_wb = app.books.open(updated_version)
    updated_ws = updated_wb.sheets(1)
    rng = updated_ws.used_range

    print(f"Used Range: {rng.address}")

    # Hightlight the rows in Excel
    for row in rng.rows:
        if row.row in highlight_rows:
            row.color = (255, 71, 76)  # light red

    updated_wb.save(Path.cwd() / "Different_Shape" / "Difference_Highlighted.xlsx")

Used Range: $A$1:$C$103
